# EECS 598 Project: Getting Started

This notebook will take you through the setup involved in the EECS 598 term project. We will also become familiar with basic SQL queries.

## 1. Install packages

Now launch Julia (either from your application launcher, or within a terminal). We're going to add some packages we will need later on. Type a closing square bracket ( `]` ) to enter pkg mode. Now type the following commands:

```julia
# type [ first to enter pkg mode (blue prompt), then do:
add LibPQ
add DataStreams
add DataFrames
add Plots
add CSV

# precompile all packages now so they load faster later:
precompile

# now backspace to exit pkg mode (prompt will return to green)
```

## 2. Connect to baseball database

The file `postgresql.jl` includes functions for connecting to the remote databases that contain baseball data:

In [ ]:
include("postgresql.jl")

We've just loaded three connection functions for you to use, depending on which database your team will be working with:

```julia
# pitch data
connect_missing_pitch(
    connection_dir,
    username, 
    password
)

# probability of an out data
connect_probability_out(
    connection_dir,
    username, 
    password
)

# pitch tunneling data
connect_pitch_tunneling(
    connection_dir,
    username,
    password
)
```

Note that each database has its own directory with unique connection files -- be careful not to enter the wrong database path! Here are the three paths to choose from:

* Probability of an out: `./postgresql-files/probability-out/`
* Pitch tunneling: `./postgresql-files/pitch-tunneling/`
* Missing pitch imputation: `./postgresql-files/missing-pitch`

Let's connect to the "missing-pitch" database as an example, using an example username and password. You will need to enter the username and password combination for your group, along with the appropriate connection directory (which contains certificates and an authentication key):

In [2]:
connection_dir = "./postgresql-files/missing-pitch"

# enter your group's username and password as strings here:
username = ??
password = ??

conn = connect_missing_pitch(connection_dir, username, password)

UndefVarError: UndefVarError: connect_missing_pitch not defined

## 3. Query databases

Note that the function printed the name of the relevant data table: `pitchmissing`. Having successfully initiated a connection, we can now send SQL queries to these tables. Let's ask for the list of columns in the `pitchmissing` table:

In [ ]:
query = """
  SELECT ordinal_position,
         column_name,
         data_type,
         numeric_precision
    FROM information_schema.columns
   WHERE table_name = 'pitchmissing'
ORDER BY ordinal_position;
"""
result = execute(conn, query)

(Here we queried a special table, the [information schema](https://www.postgresql.org/docs/9.1/static/information-schema.html). You don't need to worry about the details here -- just know that we can request metadata about tables using this schema.)

Okay, so the `execute` function returned a "PostgreSQL result" object -- but where's the data? At this point, we've told the server what we want, and it's ready to send it over. Now we ned to tell Julia to stream it in and store it somewhere locally:

In [ ]:
data = Data.stream!(result, Data.Table)

Great! Now we've streamed a bunch of data. It looks ugly though. Let's wrap it up in a nice tidy DataFrame:

In [ ]:
DataFrame(data)

Much better. We can see the list of columns in the table, ready to be queried. That was a lot of steps just to get a dataframe! That's why we included a simple function in `postgresql.jl` to make things more convenient: `stream_to_dataframe()`.

If you don't want to think about SQL anymore, we'll show you the simplest query. This next query takes all data for the whole table and streams it over. Keep in mind that this will take up a significant chunk of memory (1.3 million rows and 43 columns!), and this approach won't always work in the real world, where datasets can be much larger. Be prepared to wait a while for this cell to run -- the kernel might even crash!

In [ ]:
# WARNING: takes a long time to run! May crash your kernel!
query = """
SELECT *
FROM pitchmissing;
"""

data = stream_to_dataframe(conn, query)

In [ ]:
size(data) # pretty big

If you were able to stream the whole dataset over, you can save it locally as a CSV:

In [ ]:
using CSV, DataFrames
CSV.write("pitch-missing.csv", data)

## 4. Learn more SQL! (optional)

You probably won't need the whole dataset in your analysis. You can use SQL queries to request specific data. There are many resources out there for learning about SQL queries. [Here](https://www.w3schools.com/sql/sql_intro.asp) is a good one.

**The rest of this notebook just provides more examples of SQL queries to help you work with these large datasets.**

### Taking a subset of rows from a subset of columns

Suppose we want the first 20 rows from the "pitcherid" and "pitchtype" columns. We can set up the query, execute, and stream as follows:

In [ ]:
query = """
  SELECT pitcherid,
         pitchtype
    FROM pitchmissing
   LIMIT 20
"""

data = stream_to_dataframe(conn, query)

Why `LIMIT`? Because there are 1.3 million rows! Without LIMIT, the server would send us the entire column, and we don't need that (right now at least).

### Using GROUP BY to request an aggregated summary of data

Now let's group the data by pitcher ID and pitchtype, to see the most prolific pitchers and the kinds of throws they prefer:

In [ ]:
query = """
SELECT pitcherid, pitchtype, COUNT(pitchtype)
FROM pitchmissing
GROUP BY pitcherid, pitchtype
ORDER BY COUNT(pitchtype) DESC
LIMIT 20;
"""

data = stream_to_dataframe(conn, query)

So the pitcher with ID 434378 threw a lot of four-seam fastballs. What is the pitch speed distribution for this pitcher?

In [ ]:
query = """
SELECT pitchtype, ReleaseSpeed
FROM pitchmissing
WHERE pitcherid=434378;
"""

data = stream_to_dataframe(conn, query)

Now let's visualize this pitcher's pitch type distribution:

In [ ]:
using Plots
gr(
    label="",
    markerstrokewidth=0.3,
    markerstrokecolor="white",
    markersize=5,
)

In [ ]:
# use the project prompt pdf from Canvas to make a lookup dictionary
# for pitch types:
pitchtype_lookup = Dict(
  "CH" => "Changeup",
  "CU" => "Curveball",
  "FC" => "Cutter",
  "FF" => "Four-seam fastball",
  "FO" => "Forkball",
  "FS" => "Splitter",
  "FT" => "Two-seam fastball",
  "KC" => "Knuckle curve",
  "KN" => "Knuckleball",
  "SC" => "Screwball",
  "SI" => "Sinker",
  "SL" => "Slider"
  )

x = map(s -> pitchtype_lookup[s], data[:pitchtype])

y = data[:releasespeed]
scatter(x, y;
    xlabel="Pitch type", 
    ylabel="Speed (MPH)",
    title="Pitch speeds for pitcher 434378 ($(length(x)) pitches)"
)

### Comparing release speed, release angle, and height above plate

Let's find out more about pitch types. What are the typical measurements for a fastball vs a slider?

In [ ]:
query = """
SELECT  pitchtype,
        COUNT(pitchtype),
        AVG(ReleaseSpeed) AS ReleaseSpeed_avg,
        AVG(ReleaseAngle) AS ReleaseAngle_avg,
        AVG(TrajectoryLocationZ) AS PlateHeight_avg
FROM pitchmissing
GROUP BY pitchtype
ORDER BY ReleaseSpeed_avg DESC
"""

data = stream_to_dataframe(conn, query)

In [ ]:
pitchtype = data[:pitchtype]
releasespeed = data[:releasespeed_avg]
releaseangle = data[:releaseangle_avg]
plateheight = data[:plateheight_avg]

p1 = scatter(pitchtype, releasespeed; ylabel="Avg release speed (MPH)")
p2 = scatter(pitchtype, releaseangle; ylabel="Avg release angle (deg)")
p3 = scatter(pitchtype, plateheight; ylabel="Avg height above plate (ft)")
plot(p1, p2, p3; layout=(3, 1), size=(500, 700))

Glancing at our missing pitch prompt PDF, we see that ReleaseAngle represents the vertical angle of pitch release. The chart illustrates what we might expect: the fast a ball is released, the lower the angle should be in order for it to end up right over the home plate. You may be surprised to see that fastballs are released with a significant downward angle -- they travel so fast that they can make it all the way to the plate without dropping too far!

As a sanity check, look at the [forkball](https://en.wikipedia.org/wiki/Forkball) (FO). It has a very low release angle, and is not released very quickly, so we expect it to be quite low when it reaches the home plate. Sure enough, the bottom chart shows that forkballs end up 1.5 ft above the plate on average.

### Looking up missing values

Run the following cell to look up pitches for which the ReleaseSpeed value is missing:

In [ ]:
query = """
SELECT pitcherid, pitchtype, releasespeed, releaseangle
FROM pitchmissing
WHERE releasespeed IS NULL;
"""

data = stream_to_dataframe(conn, query)

In [ ]:
size(data)

For groups working on the pitch-missing project, these 1324 missing values are among those you will be trying to fill in! There are of course other similar commands for looking up missing values, but we have to leave some work for you to do :)